<a href="https://colab.research.google.com/github/GarganoNicolas/EVDVai_DataEngTP1/blob/main/EDVai_1_DataEng.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [186]:
import pandas as pd
import numpy as np
import re

In [187]:
path = 'https://raw.githubusercontent.com/GarganoNicolas/EVDVai_DataEngTP1/main/Utimas_Desvinculaciones.xlsx'

In [188]:
xls = pd.ExcelFile(path)

sheet_names = xls.sheet_names
print(sheet_names)

['Row data', 'Rangos', 'Managers']


In [189]:
dfs = {}

for sheet_name in sheet_names:
    dfs[sheet_name] = xls.parse(sheet_name)

data = dfs['Row data']
df_Rangos = dfs['Rangos']
df_Managers = dfs['Managers']

In [190]:
data.sample(5)

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos
3,10,Ana,Diaz,IT,2017-03-06,2020-11-30,N4,Grupo A,3,Rango 3,Rango 3,7,NO
13,6,Catalina,Gómez,Producción,2019-04-01,2022-03-31,N2,Grupo B,4,Rango 2,Rango 2,4,NO
25,20,Marcelo,Flores,Ventas,2021-09-06,2022-05-31,N2,Grupo C,6,Rango 2,Rango 2,8,SI
2,8,Francesca,López,Compras,2017-01-02,2021-06-30,N3,Grupo A,4,Rango 3,Rango 3,2,SI
19,3,Juan,Albornoz,IT,2018-06-04,2019-04-30,N4,Grupo C,4,Rango 4,Rango 4,5,NO


In [191]:
df_Rangos

,Rango,desde,hasta
0,Rango 1,150000,200000
1,Rango 2,200001,250000
2,Rango 3,250001,300000
3,Rango 4,300001,350000
4,Rango 5,350001,400000
5,Rango 6,400001,500000
6,Rango 7,500001,600000
7,Rango 8,600001,700000
8,Rango 9,700001,800000
9,Rango 10,800001,1000000


In [192]:
df_Managers

,Id,Nombre,Apellido
0,1,Rolando,Casas
1,2,Marina,Pitana
2,3,José,Feligres
3,4,Miriam,Pisani
4,5,Alejandro,Pitorino
5,6,Sonia,Kamlasky
6,7,Martín,Viglia
7,8,Ricardo,Ronaldo
8,9,Vanina,Quintana
9,10,Daniel,Quiroga


In [193]:
df_Rangos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Rango   10 non-null     object
 1   desde   10 non-null     int64 
 2   hasta   10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


In [194]:
df_Rangos[['desde', 'hasta']] = df_Rangos[['desde', 'hasta']].astype(str)
df_Rangos['pesitos'] = df_Rangos[['desde', 'hasta']].agg(' - '.join, axis=1)
df_Rangos

,Rango,desde,hasta,pesitos
0,Rango 1,150000,200000,150000 - 200000
1,Rango 2,200001,250000,200001 - 250000
2,Rango 3,250001,300000,250001 - 300000
3,Rango 4,300001,350000,300001 - 350000
4,Rango 5,350001,400000,350001 - 400000
5,Rango 6,400001,500000,400001 - 500000
6,Rango 7,500001,600000,500001 - 600000
7,Rango 8,600001,700000,600001 - 700000
8,Rango 9,700001,800000,700001 - 800000
9,Rango 10,800001,1000000,800001 - 1000000


In [195]:
# Renombrar columna para hacer un merge
df_Rangos.rename(columns={'Rango': 'Rango Salarial'}, inplace= True)
# Hacer merge
data = data.merge(df_Rangos[['Rango Salarial', 'pesitos']], on='Rango Salarial', how='left')
data[['pesitos', 'Rango Salarial']]

,pesitos,Rango Salarial
0,200001 - 250000,Rango 2
1,300001 - 350000,Rango 4
2,250001 - 300000,Rango 3
3,250001 - 300000,Rango 3
4,300001 - 350000,Rango 4
5,300001 - 350000,Rango 4
6,200001 - 250000,Rango 2
7,300001 - 350000,Rango 4
8,400001 - 500000,Rango 6
9,200001 - 250000,Rango 2


In [196]:
data.sample(3)

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos
11,1,Romina,Brito,Marketing,2018-03-05,2018-12-31,N3,Grupo B,3,Rango 3,Rango 3,1,SI,250001 - 300000
20,5,Franco,Rodríguez,Marketing,2017-05-01,2017-11-30,N5,Grupo C,6,Rango 5,Rango 4,1,SI,350001 - 400000
26,21,Cristian,Martin,Compras,2017-11-06,2020-12-31,N5,Grupo C,4,Rango 6,Rango 5,2,SI,400001 - 500000


In [197]:
Rango_salarial = data[(data['Rango Salarial'] != data['RS competencia'])]
Rango_salarial[['Id', 'Rango Salarial', 'RS competencia', 'Area']]

,Id,Rango Salarial,RS competencia,Area
4,13,Rango 4,Rango 3,IT
7,19,Rango 4,Rango 5,Auditoria
18,26,Rango 2,Rango 3,Producción
20,5,Rango 5,Rango 4,Marketing
26,21,Rango 6,Rango 5,Compras


In [198]:
df_Managers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Id        10 non-null     int64 
 1   Nombre    10 non-null     object
 2   Apellido  10 non-null     object
dtypes: int64(1), object(2)
memory usage: 368.0+ bytes


In [199]:
# Lo mismo que con Rango, pero ahora con dos columnas
df_Managers.rename(columns={'Id': 'Manager', 'Nombre': 'Nombre del Manager', 'Apellido': 'Apellido del Manager'}, inplace=True)
data = data.merge(df_Managers[['Manager', 'Nombre del Manager', 'Apellido del Manager']], on='Manager', how='left')
data.sample(3)

,Id,Nombre,Apellido,Area,Fecha comienzo,Fecha fin,Nivel,Grupo recruitment,Tiempo recruitment,Rango Salarial,RS competencia,Manager,Realizo Cursos,pesitos,Nombre del Manager,Apellido del Manager
0,4,Sebastian,González,Legales,2018-05-07,2021-05-30,N2,Grupo A,4,Rango 2,Rango 2,3,SI,200001 - 250000,José,Feligres
28,29,Dante,Martini,IT,2021-03-01,2021-12-31,N4,Grupo C,6,Rango 3,Rango 3,5,NO,250001 - 300000,Alejandro,Pitorino
7,19,Rocio,Ruiz,Auditoria,2018-02-05,2021-01-30,N4,Grupo A,4,Rango 4,Rango 5,10,NO,300001 - 350000,Daniel,Quiroga


In [200]:
# Quiero saber si se fue algun manager...
apellido = data[(data['Nombre'] == data['Nombre del Manager'])]
apellido[['Nombre', 'Apellido', 'Nombre del Manager', 'Apellido del Manager']]

,Nombre,Apellido,Nombre del Manager,Apellido del Manager


In [209]:
def remove_non_numeric(s):
    return re.sub('[^0-9]+', '', s)

# Assuming 'data' is your DataFrame and it's already defined
mean_values = data[['Area', 'Manager', 'Grupo recruitment', 'Tiempo recruitment', 'Rango Salarial', 'Nivel']]
mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(lambda x: x.apply(remove_non_numeric))
mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(pd.to_numeric, errors='coerce')

# List of column names to group by
group_columns = ['Area', 'Manager', 'Grupo recruitment']

# Dictionary to store the results for each group
group_results = {}

for column in group_columns:
    # Grouping and calculating mean values
    mean_values_group = mean_values.groupby(column)[['Tiempo recruitment', 'Rango Salarial', 'Nivel']].mean()
    mean_values_group.reset_index(inplace=True)
    mean_values_group.columns = [column, 'Mean Tiempo recruitment', 'Mean Rango Salarial', 'Mean Nivel']


    # Adding a count column for each group
    mean_values_group[f'{column}_count'] = mean_values.groupby(column)[column].transform('count')

    # Merging the mean values back to the original DataFrame
    merged_data = mean_values.merge(mean_values_group, on=column, how='left')

    # Dropping unnecessary columns and duplicates
    merged_data = merged_data.drop(['Tiempo recruitment', 'Rango Salarial', 'Nivel'], axis=1)


    # Storing the result in the dictionary
    group_results[column] = merged_data[[column, 'Mean Tiempo recruitment', 'Mean Rango Salarial', 'Mean Nivel', f'{column}_count']].drop_duplicates().sort_values(column)

# Print the results
for column, result in group_results.items():
    print(f'\n\n promedios por {column}')
    display(result)





 promedios por Area


<ipython-input-209-c97e7a9a5a5b>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(lambda x: x.apply(remove_non_numeric))
<ipython-input-209-c97e7a9a5a5b>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mean_values[['Rango Salarial', 'Nivel']] = mean_values[['Rango Salarial', 'Nivel']].apply(pd.to_numeric, errors='coerce')


,Area,Mean Tiempo recruitment,Mean Rango Salarial,Mean Nivel,Area_count
7,Auditoria,3.5,2.500000,2.500000,1
2,Compras,3.8,3.400000,3.400000,6
3,IT,4.5,3.875000,3.750000,5
0,Legales,4.0,2.000000,2.000000,8
11,Marketing,4.5,4.000000,4.000000,8
8,Producción,4.0,4.000000,4.500000,5
1,Ventas,4.5,4.166667,4.166667,8




 promedios por Manager


,Manager,Mean Tiempo recruitment,Mean Rango Salarial,Mean Nivel,Manager_count
11,1,4.5,4.000000,4.000000,1
2,2,3.8,3.400000,3.400000,6
0,3,4.0,2.000000,2.000000,5
8,4,4.5,4.000000,4.000000,3
19,5,5.4,4.400000,4.200000,3
10,6,3.5,4.000000,5.000000,5
3,7,3.0,3.000000,3.000000,3
1,8,4.5,4.166667,4.166667,2
14,9,4.5,4.000000,4.000000,2
7,10,3.5,2.500000,2.500000,5




 promedios por Grupo recruitment


,Grupo recruitment,Mean Tiempo recruitment,Mean Rango Salarial,Mean Nivel,Grupo recruitment_count
0,Grupo A,3.727273,3.636364,3.727273,11
11,Grupo B,3.625000,3.375000,3.750000,11
19,Grupo C,5.181818,4.090909,3.818182,11


In [ ]:
data['Fecha comienzo'] = pd.to_datetime(data['Fecha comienzo'])
data['Fecha fin'] = pd.to_datetime(data['Fecha fin'])
print(
    'Fecha comienzo:',
    ' min ',
    data['Fecha comienzo'].min(),
    ' max ',
    data['Fecha comienzo'].max(),
    '\nFecha fin:',
    ' min ',
    data['Fecha fin'].min(),
    ' max ',
    data['Fecha fin'].max()
    )

In [ ]:
start_date = data['Fecha comienzo'].min()
end_date = data['Fecha fin'].max()

date_range = pd.date_range(start=start_date, end=end_date, freq='M')
calendario = pd.DataFrame(date_range, columns=['Fecha']).T
calendario

In [ ]:
start_date = data['Fecha comienzo'].min()
end_date = data['Fecha fin'].max()

date_range = pd.date_range(start=start_date, end=end_date, freq='M')
calendario = pd.DataFrame(date_range, columns=['Fecha'])
calendario['Fecha'] = calendario['Fecha'].dt.strftime('%Y-%m')
print(calendario['Fecha'].dtype)
calendario.set_index('Fecha', inplace=True)
calendario=calendario.T
calendario

In [ ]:
calendario1 = calendario.copy()
calendario.insert(loc=0, column='Id', value=data['Id'])
calendario

In [ ]:
calendario.set_index('Id', inplace=True)
calendario

In [ ]:
data1 = data.copy()
data1['Fecha comienzo'] = data1['Fecha comienzo'].dt.strftime('%Y-%m')
data1['Fecha fin'] = data1['Fecha fin'].dt.strftime('%Y-%m')
selected_row = data1.loc[data1['Id'] == 4]

selected_row

In [ ]:

# Iterate over all columns in calendario
for column_name in calendario.columns:
    # Check if the value in 'Fecha comienzo' of selected_row matches the current column name
    value = 1 if selected_row['Fecha comienzo'].iloc[0] == column_name else 0

    # Update the value in the current column of calendario for the row with index 4
    calendario.at[4, column_name] = value

    # Break the iteration if the value is 1
    if value == 1:
        break

# Display the updated calendario DataFrame
calendario.head(3)



In [ ]:
data1 = data.copy()
data1['Fecha comienzo'] = data1['Fecha comienzo'].dt.strftime('%Y-%m')
data1['Fecha fin'] = data1['Fecha fin'].dt.strftime('%Y-%m')
selected_row = data1.loc[data1['Id'] == 4]

selected_row


# Flag to track the column where the value was 1
column_start = 0

# Iterate over all columns in calendario
for column_name in calendario.columns[column_start:]:
    # Check if the value in 'Fecha comienzo' of selected_row matches the current column name
    value = 1 if selected_row['Fecha comienzo'].iloc[0] == column_name else 0

    # Update the value in the current column of calendario for the row with index 4
    calendario.at[4, column_name] = value

    # Break the iteration if the value is 1
    if value == 1:
        column_start = calendario.columns.get_loc(column_name) + 1
        break

# If the value was 1, iterate from the next column
for column_name in calendario.columns[column_start:]:
    # Check the 'Fecha fin' condition
    value = 0 if selected_row['Fecha fin'].iloc[0] == column_name else 1

    # Update the value in the current column of calendario for the row with index 4
    calendario.at[4, column_name] = value

    # Break the iteration if the value is 0
    if value == 0:
        break

# After the second iteration, check if 'Fecha fin' is the same as the column name and fill with 1
if selected_row['Fecha fin'].iloc[0] == column_name:
    calendario.at[4, column_name] = 1

# Display the updated calendario DataFrame
pd.set_option('display.max_columns', None)
calendario.head(3)






In [ ]:
# Assuming 'data' is your original DataFrame and 'cacalendario' is your target DataFrame
data1 = data.copy()
data1['Fecha comienzo'] = data1['Fecha comienzo'].dt.strftime('%Y-%m')
data1['Fecha fin'] = data1['Fecha fin'].dt.strftime('%Y-%m')

# Iterate over each row in data1
for index, row in data1.iterrows():
    selected_row = row

    calendario_row_index = selected_row['Id']
    column_start = 0

    # Iterate over all columns in cacalendario
    for column_name in calendario.columns[column_start:]:
        # Check if the value in 'Fecha comienzo' of selected_row matches the current column name
        value = 1 if selected_row['Fecha comienzo'] == column_name else 0

        # Update the value in the current column of cacalendario for the row with index cacalendario_row_index
        calendario.at[calendario_row_index, column_name] = value

        # Break the iteration if the value is 1
        if value == 1:
            column_start = calendario.columns.get_loc(column_name) + 1
            break

    # If the value was 1, iterate from the next column
    for column_name in calendario.columns[column_start:]:
        # Check the 'Fecha fin' condition
        value = 0 if selected_row['Fecha fin'] == column_name else 1

        # Update the value in the current column of cacalendario for the row with index cacalendario_row_index
        calendario.at[calendario_row_index, column_name] = value

        # Break the iteration if the value is 0
        if value == 0:
            # Calculate the starting column index for the fill operation
            fill_start_index = calendario.columns.get_loc(column_name) + 1
            # Fill all the next columns with 0
            calendario.loc[calendario_row_index, calendario.columns[fill_start_index:]] = 0
            break


# Convert all columns in cacalendario to integers
calendario = calendario.astype(int)

# Display the updated cacalendario DataFrame
pd.set_option('display.max_columns', None)
calendario.head(5)


In [ ]:
calendario

In [ ]:
Trabajadores = calendario.sum().to_frame()
Trabajadores.rename(columns={0: 'n_trabajadores'}, inplace=True)
Trabajadores.T

In [ ]:
Trabajadores['relacion_altasBajas'] = Trabajadores['n_trabajadores'].diff()
Trabajadores.T

In [ ]:
altas = data['Fecha comienzo'].dt.strftime('%Y-%m')
altas.sort_values()

In [ ]:
bajas = data['Fecha fin'].dt.strftime('%Y-%m')
bajas.sort_values()

In [ ]:
Trabajadores['n_altas'] = 0
altas_counts = altas.value_counts()
for date, count in altas_counts.items():
    Trabajadores.loc[Trabajadores.index == date, 'n_altas'] = count
Trabajadores['n_bajas'] = 0
bajas_counts = bajas.value_counts()
for date, count in bajas_counts.items():
    Trabajadores.loc[Trabajadores.index == date, 'n_bajas'] = count
Trabajadores.T

In [ ]:
altas_anual = data['Fecha comienzo'].dt.strftime('%Y').value_counts()
bajas_anual = data['Fecha fin'].dt.strftime('%Y').value_counts()

# Convert each Series to a DataFrame and reset the index
altas_anual_df = altas_anual.to_frame(name='altas').reset_index()
bajas_anual_df = bajas_anual.to_frame(name='bajas').reset_index()

# Rename the index column to 'Año'
altas_anual_df.rename(columns={'index': 'Año'}, inplace=True)
bajas_anual_df.rename(columns={'index': 'Año'}, inplace=True)

# Merge the two DataFrames on 'Año', using a left join to include all years from altas_anual
relacion_altas_bajas_anual = pd.merge(altas_anual_df, bajas_anual_df, on='Año', how='left')

# Calculate the difference between altas and bajas and add it as a new column
relacion_altas_bajas_anual['dif'] = relacion_altas_bajas_anual['altas'] - relacion_altas_bajas_anual['bajas']

# Append a final row with the total counts of altas and bajas
total_altas = altas_anual.sum()
total_bajas = bajas_anual.sum()
total_dif = total_altas - total_bajas
relacion_altas_bajas_anual = relacion_altas_bajas_anual.append({'Año': 'Total', 'altas': total_altas, 'bajas': total_bajas, 'dif': total_dif}, ignore_index=True)

relacion_altas_bajas_anual.sort_values('Año')


In [ ]:
data